## Installation

In [ ]:
!pip install relevanceai pandas redis tqdm vecdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 9.9 MB/s eta 0:00:00


## Instantiate Relevance AI and set Redis up

You can set the connection string via the dashboard instead as well. Setting keys in the python code requires an admin permission API key

In [ ]:
import relevanceai as rai
rai.login()

You can create and find your API key in your browser here: https://chain.relevanceai.com/login/sdk
Paste the API key from your profile and hit enter: ··········
Paste the Region from your profile and hit enter: f1db6c
Paste the Project from your profile and hit enter: 32e34138-16b0-4d62-93a6-1921ccadc1a1
Successfully logged in, welcome!


Set `redis://read@Llmstack1!@redis-16021.c24680.us-east-1-1.ec2.cloud.rlrcp.com:16021` for redis connection string

In [ ]:
from relevanceai.steps import RedisSearch

step = RedisSearch(
    index="mlopscommunity",
    query="hello",
    vector_field="vector",
    model="text-embedding-ada-002",
    page_size=5
)

In [ ]:
results = step.run()

In [ ]:
results["results"][0]["message"]

'U016HB7Q5A6: Hi here :wave:UP3T8K9M5: Hey hey!'

## Moving it into a chain

Specify the parameters of the chains. I.e. whats the input of the chain.

The parameter can be specified in any inputs into a step inside a chain via "{{params.something}}"

In [ ]:
from relevanceai.params import StringParam
sp = StringParam("question", "Question")

In [ ]:
chain = rai.create(
    name="Q&A on Chat messages",
    description="",
    parameters=sp
)
chain.reset()

In [ ]:
search_step = RedisSearch(
    index="mlopscommunity",
    query="{{params.question}}",
    vector_field="vector",
    model="text-embedding-ada-002",
    page_size=5
)

chain.add(search_step)

Every step has outputs. If you want to feed an output into a step further down the chain just put "{{ }}" around it. You can also that value with jsonpath

In [ ]:
search_step.outputs

['steps.redis_search.output.results']

In [ ]:
from relevanceai.steps import PromptCompletion
llm_qa = PromptCompletion(
    prompt="Context:{{steps.redis_search.output.results[*].message}}\nBased on the above context answer the question.Question:{{params.question}}\nAnswer:",
)
chain.add(llm_qa)

In [ ]:
chain.run({
    "question" : "What is a vector database?"
})['answer']

'A vector database is a type of database used in machine learning that stores vector representations of data and allows for fast querying and searching of that data. It can also handle versioning and updating of embeddings.'

Once the chain is good you can deploy it as an API to use anywhere from our GUI, Javascript or just as an API request




In [ ]:
chain.deploy()

Studio deployed successfully to id  6fac8317-0777-43f8-8910-bb9f72554256
Your studio id is randomly generated, to ensure you are updating the same chain you should specify the id on rai.create(id=id) 

=============Low Code Notebook================
You can share/visualize your chain as an app in our low code notebook here: https://chain.relevanceai.com/notebook/f1db6c/32e34138-16b0-4d62-93a6-1921ccadc1a1/6fac8317-0777-43f8-8910-bb9f72554256/app

=============with Requests================
Here is an example of how to run the chain with API: 

import requests
requests.post(https://api-f1db6c.stack.tryrelevance.com/latest/studios/6fac8317-0777-43f8-8910-bb9f72554256/trigger_limited", json={
    "project": "32e34138-16b0-4d62-93a6-1921ccadc1a1",
    "params": {
        YOUR PARAMS HERE
    }
})
            

=============with Python SDK================
Here is an example of how to run the chain with Python: 

import relevanceai as rai
chain = rai.load("6fac8317-0777-43f8-8910-bb9f72554256"

'6fac8317-0777-43f8-8910-bb9f72554256'